In [36]:
from dquartic.utils import data_loader
from dquartic.model.unet1d import UNet1d
from dquartic.utils.data_loader import DIAMSDataset
import torch
import torch.quantization
import torch
from torch.utils.data import Dataset, DataLoader


In [35]:

ms1_file = "/home/nuc11extreme-012/Downloads/diffusion-deconvolution-dia-msms-data/josh_gpf/ms1_data_int32.npy"
ms2_file = "/home/nuc11extreme-012/Downloads/diffusion-deconvolution-dia-msms-data/josh_gpf/ms2_data_cat_int32.npy"


In [37]:
dataset = DIAMSDataset(ms1_file=ms1_file, ms2_file=ms2_file, normalize="minmax")
data_loader = DataLoader(dataset, batch_size=1, shuffle=True, num_workers=1)
data_iter = iter(data_loader)
ms2_1, ms1_1, ms2_2,ms1_2 = next(data_iter)


Info: Loaded  520 MS2 slice samples and 520 MS1 slice samples from NPY files.


In [6]:
model_init={
              "dim": 4,
      "channels": 1,
      "dim_mults": [
        1,
        2,
        2,
        2,
        4,
        4,
        4
      ],
      "conditional": True,
      "init_cond_channels": 1,
      "attn_cond_channels": 1,
      "tfer_dim_mult": 620,
      "downsample_dim": 40000,
      "simple": True
}
device = torch.device("cpu")
model = UNet1d(**model_init).to(device)

In [30]:
checkpoint = torch.load("/home/nuc11extreme-012/Downloads/diffusion-deconvolution-dia-msms-data/best_model.pth", map_location=torch.device('cpu'))
state_dict = checkpoint["model_state_dict"]


/tmp/ipykernel_51189/4080633071.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load("/home/nuc11extreme-012/Downloads/diffusion-deconvolution-dia-msm

In [32]:
# The 'fbgemm' backend is generally used for x86 architectures.
model.qconfig = torch.quantization.get_default_qconfig('fbgemm')
model.eval()
# 4. Prepare the model for static quantization
torch.quantization.prepare(model, inplace=True)


/home/nuc11extreme-012/anaconda3/envs/dquartic/lib/python3.9/site-packages/torch/ao/quantization/observer.py:221: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a future release of PyTorch.
  warnings.warn(


UNet1d(
  (init_conv): QuantizedConv1d(2, 4, kernel_size=(7,), stride=(1,), scale=1.0, zero_point=0, padding=(3,))
  (time_mlp): Sequential(
    (0): SinusoidalPosEmb()
    (1): QuantizedLinear(in_features=4, out_features=16, scale=0.012041483074426651, zero_point=62, qscheme=torch.per_channel_affine)
    (2): GELU(approximate='none')
    (3): QuantizedLinear(in_features=16, out_features=16, scale=0.02822525054216385, zero_point=46, qscheme=torch.per_channel_affine)
  )
  (init_cond_proj): ConditionalScaleShift(
    (to_scale_shift): Sequential(
      (0): SiLU()
      (1): QuantizedLinear(in_features=16, out_features=2, scale=0.04809444397687912, zero_point=0, qscheme=torch.per_channel_affine)
    )
  )
  (attn_cond_proj): ModuleList(
    (0): Identity()
    (1): Sequential(
      (0): QuantizedConv1d(1, 8, kernel_size=(7,), stride=(1,), scale=1.0, zero_point=0, padding=(3,))
      (1): GELU(approximate='none')
      (2): QuantizedConv1d(8, 8, kernel_size=(1,), stride=(1,), scale=1.0,

In [33]:
calibration_loader = data_loader

In [16]:
for i, batch in enumerate(calibration_loader):
    ms2_1, ms1_1, ms2_2,ms1_2 = batch[0], batch[1], batch[2], batch[3]
    print(ms2_1.shape)
    print(ms1_1.shape)
    print(ms2_2.shape)
    print(ms1_2.shape)
    break

torch.Size([1, 34, 40000])
torch.Size([1, 34])
torch.Size([1, 34, 40000])
torch.Size([1, 34])


In [34]:
# Modify your calibration code to match the expected input format
with torch.no_grad():
    for i, batch in enumerate(calibration_loader):
        try:
            ms2_1 = batch[0].to(device)
            ms1_1 = batch[1].to(device)
            time = torch.ones(1, dtype=torch.float32).to(device) #wasnt sure if this was correct but we can consider it for testing with sample time tensor input

            model(ms2_1, time, ms1_1) #needs verification
            print(f"Successfully processed batch {i}")
            if i >= 5:
                break
        except Exception as e:
            print(f"Error in batch {i}: {e}")
            continue

# After calibration, convert to quantized model
torch.quantization.convert(model, inplace=True)
    
    # Save the quantized model
torch.save(model.state_dict(), "model_quantized.pth")
print("Static quantization complete. Quantized model saved as 'model_quantized.pth'.")

Error in batch 0: Could not run 'quantized::linear' with arguments from the 'CPU' backend. This could be because the operator doesn't exist for this backend, or was omitted during the selective/custom build process (if using custom build). If you are a Facebook employee using PyTorch on mobile, please visit https://fburl.com/ptmfixes for possible resolutions. 'quantized::linear' is only available for these backends: [Meta, QuantizedCPU, QuantizedCUDA, BackendSelect, Python, FuncTorchDynamicLayerBackMode, Functionalize, Named, Conjugate, Negative, ZeroTensor, ADInplaceOrView, AutogradOther, AutogradCPU, AutogradCUDA, AutogradXLA, AutogradMPS, AutogradXPU, AutogradHPU, AutogradLazy, AutogradMeta, Tracer, AutocastCPU, AutocastXPU, AutocastCUDA, FuncTorchBatched, BatchedNestedTensor, FuncTorchVmapMode, Batched, VmapMode, FuncTorchGradWrapper, PythonTLSSnapshot, FuncTorchDynamicLayerFrontMode, PreDispatch, PythonDispatcher].

Meta: registered at ../aten/src/ATen/core/MetaFallbackKernel.cpp:

In [ ]:
dynamic_quantized_model = torch.quantization.quantize_dynamic(
    model, {torch.nn.Linear}, dtype=torch.qint8
)
print("Dynamic quantization applied.")

# torch.save(dynamic_quantized_model.state_dict(), 'dynamic_quantized_model.pth')
